In [ ]:
import datasets
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import openai
from sklearn.model_selection import train_test_split
import json
import time 

In [ ]:
openai.api_key = "Enter Your API Key"

In [ ]:
df, test_df = pd.read_csv('Amazon_Reviews_Train.csv'), pd.read_csv('Amazon_Reviews_Test.csv')

In [ ]:
num_labels = 5
id_to_label = {
    0: "Bad",
    1: "Good",
    2: "Neutral",
    3: "Very Good",
    4: "Excellent"
}

label_to_id = {value : key for key , value in  id_to_label.items()}


In [ ]:
df['reviews.rating'] = df['reviews.rating'].map(id_to_label)
test_df['reviews.rating'] = test_df['reviews.rating'].map(id_to_label)

In [ ]:
df

In [ ]:
categories = df['reviews.rating'].unique()

In [ ]:
categories

In [ ]:
def text_to_openai_json(data,filename):
    """
    Converts a given dataset into a JSON Lines (JSONL) file suitable for OpenAI's GPT-3.5 turbo model.
    
    Args:
        data (DataFrame or similar data structure): Input data containing text and labels.

    The function processes the input data row by row, constructing conversations for each row with a system message, user message, and an assistant message. It then writes the generated conversation data to a JSONL file.
 
    """
    # Initialize an empty list to store conversation data
    message_list = []

    # Iterate through the rows in the input data
    for _, row in data.iterrows():
        # Create a system message as an initial instruction
        system_message = {
            "role": "system",
            "content": f"given the following text: find the category in: {categories} that is most closely associated with it. Return only the category name"
        }

        # Append the system message to the conversation
        message_list.append({"messages": [system_message]})

        # Create a user message based on the 'text' column from the data
        user_message = {
            "role": "user",
            "content": row['reviews.text']
        }

        # Append the user message to the conversation
        message_list[-1]["messages"].append(user_message)

        # Create an assistant message based on the 'coarse_label' column from the data
        assistant_message = {
            "role": 'assistant',
            "content": row['reviews.rating']
        }

        # Append the assistant message to the conversation
        message_list[-1]["messages"].append(assistant_message)

    # Write the conversation data to a JSON Lines (JSONL) file
    with open(filename, "w") as json_file:
        for message in message_list:
            # Serialize the conversation data to JSON and write it to the file
            json.dump(message, json_file)
            json_file.write("\n")

In [ ]:
system_content =  f"given the following text: find the category in: {categories} that is most closely associated with it. Return only the category name only in following format"

In [ ]:
system_content

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
def zero_shot_model(data,model_id):
    pred = []
    for row in data["reviews.text"]:
        completion = openai.ChatCompletion.create(
            model= model_id ,
            messages=[
                {"role": "system", "content": system_content },
                {"role": "user", "content": row }
            ])
        
        print(f'text: {row}')
        print(completion.choices[0].message.content)
        pred.append(completion.choices[0].message.content)
    pred_df = pd.DataFrame({'reviews.text': data["reviews.text"], 'reviews.rating' : data['reviews.rating'], 'few-shot predictions' : pred })
    
    return pred_df

In [ ]:
cumulative_increment = 0
model_id = 'gpt-3.5-turbo-0301'
all_model_id = []

In [ ]:
def fine_tune_model(model_id,num_label,pred_df):
    incorrection_pred_df = pred_df[pred_df['reviews.rating'] != pred_df['few-shot predictions']][:num_label]
    filename = f'ft_increment_{num_label}.jsonl'
    text_to_openai_json(incorrection_pred_df, filename)
    loader = openai.File.create(file=open(filename, "rb"), purpose='fine-tune')
    fine_tuning_job = openai.FineTuningJob.create(training_file=loader.id, model="gpt-3.5-turbo")
    return fine_tuning_job.id

In [ ]:
def ft_accuracy(data,model_id):
    pred = []
    for row in data["reviews.text"]:
        completion = openai.ChatCompletion.create(
            model= model_id ,
            messages=[
                {"role": "system", "content": system_content },
                {"role": "user", "content": row }
            ])
        
        print(f'text: {row}')
        print(completion.choices[0].message.content)
        pred.append(completion.choices[0].message.content)
        
    accuracy = accuracy_score(data['reviews.rating'], pred)
    print(f'Accuracy: {accuracy * 100:.2f}%')
    precision, recall, f1, _ = precision_recall_fscore_support(data['reviews.rating'], pred, average='macro',zero_division=1)
    
    return accuracy, precision, recall, f1, pred

In [ ]:
test_df.head()

In [ ]:
pred_df = zero_shot_model(model_id='gpt-3.5-turbo-0301', data = test_df[:400])

In [ ]:
def wait_for_fine_tuning(job_id):
    while True:
        response = openai.FineTuningJob.retrieve(job_id)
        print(response["fine_tuned_model"])
        if response["fine_tuned_model"]:
            print(response["fine_tuned_model"])
            return response["fine_tuned_model"]
        time.sleep(60)  # Check every 60 seconds

In [ ]:
model_ids = []
accs = [] 
precisions = [] 
recalls = [] 
f1s = []
labels = []
label_count = []
count = 0
for i in range(15):
    count += 10
    label_count.append(count)
    ft_id = fine_tune_model(model_id = 'gpt-3.5-turbo-0301', num_label=label_count, pred_df=pred_df)
    if wait_for_fine_tuning(ft_id) is not None:
        model_ids.append(wait_for_fine_tuning(ft_id))
        accuracy, precision, recall, f1, pred = ft_accuracy(data=test_df[:400],model_id=(wait_for_fine_tuning(ft_id)))
        labels.append(label_count)
        accs.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)

In [ ]:
accs

# Confusion Matrix


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
cm = confusion_matrix( test_df['reviews.rating'][:400],pred)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix for FT3.5 GPT Amazon Label Prediction')
plt.show()

In [ ]:
data = ({ 'Pred':  pred,'Actual':  test_df['reviews.rating'][:400]})

In [ ]:
pd.DataFrame(data).to_csv('FT-GPT3.5_turbo_amazon_pred.csv',index=False)